FIle.csv
File 
    method
    method
    
    
Readme

1) utils -> methode pour recuper tt les fichiers.py
2) utils -> toutes les noms de functions
3) main :
    loop 
        File
         method

In [ ]:
pip install openai
pip install ast

In [ ]:
import os
import ast

def list_python_files(path):
    """
    List all Python files in the given directory path.
    """
    python_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files


def get_functions_definitions(file_path):
    with open(file_path, 'r') as file:
        source_code = file.read()
        tree = ast.parse(source_code)
        function_definitions = []

        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                function_code = ast.get_source_segment(source_code, node)
                function_definitions.append(function_code)

        return function_definitions




In [ ]:
list_python_files("workshop-docs-agent")


In [ ]:
mainframe = get_functions_definitions("workshop-docs-agent/resources/IIRA/gui/mainframe.py")
print(mainframe)